# Bài 1:  Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức

In [ ]:
import sqlite3
import math

# Kết nối đến SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
    CREATE TABLE data_table (
        A REAL,
        B REAL
    )
''')

# Chèn dữ liệu mẫu vào bảng
data = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]
cursor.executemany("INSERT INTO data_table (A, B) VALUES (?, ?)", data)
conn.commit()

# Truy vấn dữ liệu
query = '''
SELECT 
    COUNT(*) AS n,
    SUM(A) AS sum_a,
    SUM(B) AS sum_b,
    SUM(A * B) AS sum_ab,
    SUM(A * A) AS sum_a2,
    SUM(B * B) AS sum_b2
FROM data_table;
'''

cursor.execute(query)
n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = cursor.fetchone()

# Tính hệ số tương quan Pearson 
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt((n * sum_a2 - sum_a**2) * (n * sum_b2 - sum_b**2))
correlation = numerator / denominator if denominator != 0 else None

print("Hệ số tương quan Pearson:", correlation)

# Đóng kết nối
cursor.close()
conn.close()

Hệ số tương quan Pearson: 1.0


Hệ số tương quan 1.0 cho thấy mối tương quan tuyến tính hoàn hảo và dương. Khi 1 biến tăng biến còn lại cũng sẽ tăng

# Bài 2:  Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2

In [8]:
import scipy.stats as stats

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
    CREATE TABLE car_scores (
        day TEXT,
        model TEXT,
        score REAL
    )
''')

data = [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany("INSERT INTO car_scores (day, model, score) VALUES (?, ?, ?)", data)
conn.commit()

# Truy vấn dữ liệu
query = '''
SELECT * FROM car_scores
'''
df = pd.read_sql_query(query, conn)

# Chuyển đổi dữ liệu sang pivot table
pivot_table = df.pivot(index="day", columns="model", values="score")

# Thực hiện kiểm định Chi-Square (X^2)
chi2, p_value, dof, expected = stats.chi2_contingency(pivot_table)

# Hiển thị kết quả
print("Bảng dữ liệu:\n", pivot_table)
print("\nThống kê Chi-Square:", chi2)
print("Giá trị P:", p_value)
print("Bậc tự do:", dof)
print("Tần suất kỳ vọng:\n", expected)

# Đóng kết nối
cursor.close()
conn.close()

Bảng dữ liệu:
 model    A    B    C
day                 
Day 1  8.0  9.0  7.0
Day 2  7.5  8.5  7.0
Day 3  6.0  7.0  8.0
Day 4  7.0  6.0  5.0

Thống kê Chi-Square: 0.826439691310499
Giá trị P: 0.9913459315666546
Bậc tự do: 6
Tần suất kỳ vọng:
 [[7.95348837 8.51162791 7.53488372]
 [7.62209302 8.15697674 7.22093023]
 [6.95930233 7.44767442 6.59302326]
 [5.96511628 6.38372093 5.65116279]]


Giá trị P rất cao (gần 1), cho thấy không có sự khác biệt có ý nghĩa thống kê giữa các mô hình A, B, và C. Nói cách khác, hiệu suất của ba mô hình không khác biệt đáng kể về mặt thống kê, mặc dù có sự dao động trong dữ liệu.

Tần suất độ vòng cho thấy sự phân bố không đồng đều. Các độ vòng 6 và 7 chiếm ưu thế, trong khi độ vòng 5 và 8 ít xuất hiện hơn.
Điều này có thể phản ánh rằng hiệu suất của các mô hình thường dao động ở mức trung bình (6, 7), hiếm khi đạt mức cao (8) hoặc thấp (5), phù hợp với dữ liệu điểm số.

# Bài 3: Bảng flights (departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian

In [9]:
# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute("CREATE TABLE flights (id INTEGER PRIMARY KEY, departure_time INTEGER)")

# Chèn dữ liệu mẫu
data = [(1, 830), (2, 1445), (3, 1230), (4, 700), (5, 2200)]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", data)
conn.commit()

# Truy vấn và chuyển đổi thời gian
query = """
SELECT id, departure_time,
       printf('%02d:%02d %s', 
              departure_time / 100 % 12, 
              departure_time % 100, 
              CASE WHEN departure_time / 100 >= 12 THEN 'PM' ELSE 'AM' END
       ) AS formatted_time
FROM flights
"""
cursor.execute(query)

# Hiển thị kết quả
for row in cursor.fetchall():
    print(row)

# Đóng kết nối
conn.close()

(1, 830, '08:30 AM')
(2, 1445, '02:45 PM')
(3, 1230, '00:30 PM')
(4, 700, '07:00 AM')
(5, 2200, '10:00 PM')


Chuyển đổi sang định dạng 12h để dễ hiểu và dễ dàng quản lí thời gian hơn

# Bài 4:  Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [11]:
# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
# Tạo bảng dữ liệu
cursor.execute("CREATE TABLE data (id INTEGER PRIMARY KEY, value REAL)")

# Thêm dữ liệu mẫu
data_values = [(1, 10), (2, 12), (3, 15), (4, 14), (5, 11), 
               (6, 13), (7, 50), (8, 14), (9, 12), (10, 13)]
cursor.executemany("INSERT INTO data (id, value) VALUES (?, ?)", data_values)
conn.commit()

# Truy vấn tính toán MAD và phát hiện ngoại lệ
query = """
WITH median_calc AS (
    SELECT value 
    FROM data
    ORDER BY value
    LIMIT 1 OFFSET (SELECT COUNT(*) / 2 FROM data)
),
stats AS (
    SELECT 
        value,
        ABS(value - (SELECT value FROM median_calc)) AS deviation
    FROM data
),
mad_table AS (
    SELECT AVG(deviation) AS mad FROM stats
)
SELECT d.id, d.value 
FROM data d, mad_table, median_calc
WHERE ABS(d.value - median_calc.value) > 1.5 * mad_table.mad;
"""
cursor.execute(query)

# Hiển thị ngoại lệ
outliers = cursor.fetchall()
print("Ngoại lệ:", outliers)

# Đóng kết nối
conn.close()

Ngoại lệ: [(7, 50.0)]


Giá trị 50.0 là một ngoại lệ rõ rệt, vì nó khác biệt lớn so với phần còn lại của tập dữ liệu (các giá trị khác dao động từ 10 đến 15). Điều này có thể do lỗi nhập liệu, sự kiện bất thường, hoặc một đặc điểm đặc biệt của dữ liệu tại ID 7. Cần kiểm tra thêm để xác định nguyên nhân và quyết định cách xử lý (loại bỏ, điều chỉnh, hoặc giữ lại nếu có ý nghĩa đặc biệt).

# Bài 5: Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [16]:
# Kết nối đến cơ sở dữ liệu SQLite 
conn = sqlite3.connect('patients.db')
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
CREATE TABLE IF NOT EXISTS Patient (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    weight REAL,
    height REAL
)
''')

# Chèn một số dữ liệu mẫu vào bảng Patient
cursor.execute('''
INSERT OR IGNORE INTO Patient (last_name, weight, height)
VALUES ('Nguyen', 70, 175),
       ('Tran', 65, 160),
       ('Nguyen', 70, 175),
       ('Le', 80, 180)
''')

# Thực hiện truy vấn SQL để so sánh các bệnh nhân
cursor.execute('''
SELECT p1.id AS patient_id1, p2.id AS patient_id2,
       CASE 
           WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight THEN 'Cả hai là một người'
           ELSE 'Đây là hai người khác nhau'
       END AS comparison_result
FROM Patient p1, Patient p2
WHERE p1.id < p2.id
''')

results = cursor.fetchall()
for row in results:
    print(f"Patient {row[0]} vs Patient {row[1]}: {row[2]}")

# Đóng kết nối
conn.commit() 
conn.close()

Patient 1 vs Patient 2: Đây là hai người khác nhau
Patient 1 vs Patient 3: Cả hai là một người
Patient 1 vs Patient 4: Đây là hai người khác nhau
Patient 2 vs Patient 3: Đây là hai người khác nhau
Patient 2 vs Patient 4: Đây là hai người khác nhau
Patient 3 vs Patient 4: Đây là hai người khác nhau


Truy vấn đã xác định chính xác cặp bệnh nhân 1 và 3 có khả năng là cùng một người dựa trên tiêu chí last_name và weight. Điều này có thể hữu ích trong việc phát hiện dữ liệu trùng lặp.